Set Parameters

In [ ]:
similarity_cutoff = 0.05 

Import files 

In [ ]:
import xarray as xr 

In [ ]:
compound = xr.open_dataset('compound_nicholas_short.nc', chunks={'time': 'auto'})
surge = xr.open_dataset('surge_nicholas_short.nc', chunks={'time': 'auto'})
#rivers = xr.open_dataset('river_nicholas_short.nc', chunks={'time':'auto'})

In [ ]:
x = compound['x']; y = compound['y'] 
ele_X_nodes = compound['element']-1 #extract element to global node map (connectivity map)

compound_zeta = compound['zeta']
surge_zeta = surge['zeta']

Calculate Differences

In [ ]:
from numba import vectorize, float64
import numpy as np 

#Need to formalize this logic. 
@vectorize([float64(float64,float64)])
def bathymetric_rel_diff(a: float, b: float) -> float:
    """
    Takes a relative-to-minuend difference between two bathymetries, making optimizations where dry cells are 
    encountered. Intended to be combined with Numba's vectorize and Xarray's apply_ufunc.                                                                                                                        

    Parameters
    ----------
    a : np.ndarray
        Minuend. 
    b : np.ndarray
        Subtrahend. 
    
    Returns
    -------
    difference : np.float
    
    Examples
    --------
    >>> a = [1 nan nan]
    >>> b = [1 1 nan]
    >>> xr.apply_ufunc(fast_bathymetric_diff_atomic,a,b)
    [0 -1 0]
    
    """    
    #Case where both are dry 
    if np.isnan(a) and np.isnan(b): 
        return 0
    
    #Double check this behavior. 
    if np.isnan(a): 
        return 999 
    if np.isnan(b): 
        b = 0 
    
    return abs((a-b)/a) 

compound_surge_similarity = xr.apply_ufunc(bathymetric_rel_diff, compound_zeta, surge_zeta, dask ="parallelized")

Mesh data

In [ ]:
import geoviews as gv 
import holoviews as hv 
from holoviews.operation.datashader import rasterize
import datashader as ds

times = compound['time']

points = gv.operation.project_points(gv.Points((x,y)))

tris = ele_X_nodes.to_numpy()

def mesh_at_time(time): 
    depth_points = points.add_dimension('zeta', 0, compound_surge_similarity[time], vdim = True)
    return gv.TriMesh((tris, depth_points))

meshes_dict = {t:mesh_at_time(t) for t in times}
meshes_HoloMap = hv.HoloMap(meshes_dict, kdims="times")

tiles = gv.tile_sources.OSM 
meshes_raster = rasterize(meshes_HoloMap, interpolation='linear', aggregator=ds.mean('zeta')).opts(colorbar=True, color_levels=[0,similarity_cutoff,np.inf], cmap=['#5ebaff','#ff8f20'])
plot = tiles * meshes_raster 

hv.output(plot, holomap='scrubber', fps = 4)
